# NEWS

In [1]:

# BigQuery is pre-installed on Colab, but just in case:
# !pip install google-cloud-bigquery pandas-gbq db-dtypes

from google.colab import auth
auth.authenticate_user()  # This opens a popup - sign in with your Google account

print("✅ Authenticated!")

✅ Authenticated!


In [6]:
# CELL 1: Create YOUR OWN project (unique name)
!gcloud projects create resilience-ai-aatu --name="ResilienceAI"

Create in progress for [https://cloudresourcemanager.googleapis.com/v1/projects/resilience-ai-aatu].
Enabling service [cloudapis.googleapis.com] on project [resilience-ai-aatu]...
Operation "operations/acat.p2-130550627089-15e34159-cdcc-482f-9fe4-f56de529ee66" finished successfully.


In [8]:
# CELL 2: Set it as active and enable BigQuery
!gcloud config set project resilience-ai-aatu
!gcloud services enable bigquery.googleapis.com

Updated property [core/project].


In [9]:
# CELL 3: Now run the query
PROJECT_ID = "resilience-ai-aatu"
df = pd.read_gbq(query, project_id=PROJECT_ID, dialect='standard')

/tmp/ipython-input-3962447148.py:3: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id=PROJECT_ID, dialect='standard')


In [4]:
!gcloud projects list

PROJECT_ID                  NAME              PROJECT_NUMBER  ENVIRONMENT
aatmajisro                  aatmajisro        416402854081
auditbot-485520             AuditBOT          1054850965882
gen-lang-client-0033587703  BGAai             208171978500
gen-lang-client-0049178312  DASH              203307360027
gen-lang-client-0237377083  Gemini API        38382449363
moonlit-balm-445410-v6      My First Project  578876382740
phonic-agility-445410-m5    My First Project  1058567967140
resilience-ai-487422        resilience-ai     755660076598
travel-recom-445302         Travel-Recom      29683591273
void-ai-486700              Void AI           540362233072


To take a quick anonymous survey, run:
  $ gcloud survey



In [10]:

# ---- CELL 3: Run the Query ----

import pandas as pd

# This is the main query - gets monthly country-level disruption data
# from 2000 to 2024. Covers ALL countries.
# Estimated query size: ~5-8 GB (well within free 1TB limit)

query = """
SELECT
    ActionGeo_CountryCode AS country_code,
    CAST(FLOOR(SQLDATE / 100) AS INT64) AS year_month,
    CAST(Year AS INT64) AS year,
    CAST(FLOOR(MOD(SQLDATE, 10000) / 100) AS INT64) AS month,

    -- Total event counts
    COUNT(*) AS total_events,

    -- WAR & ARMED CONFLICT events
    COUNTIF(EventRootCode IN ('18','19','20')) AS war_events,

    -- PROTEST & POLITICAL INSTABILITY
    COUNTIF(EventRootCode = '14') AS protest_events,

    -- SANCTIONS, EMBARGOES, COERCION (trade disruptions)
    COUNTIF(EventRootCode IN ('16','17')) AS sanctions_coercion_events,

    -- HUMANITARIAN AID (indicates crisis/disaster happened)
    COUNTIF(EventRootCode = '07') AS humanitarian_aid_events,

    -- THREATS (escalation signals)
    COUNTIF(EventRootCode = '13') AS threat_events,

    -- DEMANDS & DISAPPROVAL (diplomatic tension)
    COUNTIF(EventRootCode IN ('10','11','12')) AS diplomatic_tension_events,

    -- FORCE POSTURE (military mobilization)
    COUNTIF(EventRootCode = '15') AS force_posture_events,

    -- QuadClass breakdown (CAMEO classification)
    COUNTIF(QuadClass = 1) AS verbal_cooperation,
    COUNTIF(QuadClass = 2) AS material_cooperation,
    COUNTIF(QuadClass = 3) AS verbal_conflict,
    COUNTIF(QuadClass = 4) AS material_conflict,

    -- Severity metrics
    AVG(GoldsteinScale) AS avg_goldstein_scale,
    MIN(GoldsteinScale) AS min_goldstein_scale,
    MAX(GoldsteinScale) AS max_goldstein_scale,

    -- Media sentiment & attention
    AVG(AvgTone) AS avg_tone,
    MIN(AvgTone) AS min_tone,
    SUM(NumMentions) AS total_mentions,
    SUM(NumArticles) AS total_articles,
    SUM(NumSources) AS total_sources,

    -- Severe events (Goldstein < -5 means serious conflict)
    COUNTIF(GoldsteinScale < -5) AS severe_negative_events,
    COUNTIF(GoldsteinScale < -8) AS extreme_negative_events,

    -- Material conflict severity
    AVG(CASE WHEN QuadClass = 4 THEN GoldsteinScale END) AS avg_material_conflict_severity

FROM `gdelt-bq.full.events`
WHERE Year >= 2000 AND Year <= 2024
    AND ActionGeo_CountryCode IS NOT NULL
    AND ActionGeo_CountryCode != ''
GROUP BY country_code, year_month, year, month
ORDER BY year_month, country_code
"""

print("Running BigQuery... (this takes 1-3 minutes)")
print("Query processes ~5-8 GB of GDELT data (free tier = 1 TB/month)")

df = pd.read_gbq(query, project_id=PROJECT_ID, dialect='standard')

print(f"\n✅ Done! Shape: {df.shape}")
print(f"   Countries: {df['country_code'].nunique()}")
print(f"   Date range: {df['year_month'].min()} to {df['year_month'].max()}")
print(f"   Total rows: {len(df):,}")

Running BigQuery... (this takes 1-3 minutes)
Query processes ~5-8 GB of GDELT data (free tier = 1 TB/month)


/tmp/ipython-input-3321541899.py:76: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id=PROJECT_ID, dialect='standard')



✅ Done! Shape: (72748, 27)
   Countries: 266
   Date range: 200001 to 202412
   Total rows: 72,748


In [12]:
print("\n📊 Sample Data:")
print(df.head(10))

print("\n📊 Column Types:")
print(df.dtypes)

print("\n📊 Basic Stats:")
print(df.describe())

print("\n📊 Top 10 Countries by Conflict Events:")
top_conflict = df.groupby('country_code')['war_events'].sum().sort_values(ascending=False).head(10)
print(top_conflict)


📊 Sample Data:
  country_code  year_month  year  month  total_events  war_events  \
0           AC      200001  2000      1            19           4   
1           AE      200001  2000      1           489          19   
2           AF      200001  2000      1          2612         198   
3           AG      200001  2000      1          2446         302   
4           AJ      200001  2000      1          2061          59   
5           AL      200001  2000      1          2715         291   
6           AM      200001  2000      1          1369          82   
7           AO      200001  2000      1          2710         405   
8           AQ      200001  2000      1             4           0   
9           AR      200001  2000      1          1227          70   

   protest_events  sanctions_coercion_events  humanitarian_aid_events  \
0               0                          2                        0   
1               0                         17                        6   
2    

In [13]:

# ---- CELL 5: Add Derived Features ----

# These features make the data more useful for your ML models

# Conflict ratio: what % of events are conflict?
df['conflict_ratio'] = (df['war_events'] + df['material_conflict']) / df['total_events'].clip(lower=1)

# Instability index: weighted score of all disruption types
df['instability_index'] = (
    df['war_events'] * 4 +
    df['protest_events'] * 2 +
    df['sanctions_coercion_events'] * 3 +
    df['threat_events'] * 1.5 +
    df['force_posture_events'] * 2.5
) / df['total_events'].clip(lower=1)

# Tone deviation (how negative is sentiment vs neutral?)
df['tone_negativity'] = df['avg_tone'].apply(lambda x: abs(min(0, x)))

# Media attention intensity (log scale)
import numpy as np
df['media_attention_log'] = np.log1p(df['total_articles'])

# Month-over-month change in conflict (useful for trend detection)
df = df.sort_values(['country_code', 'year_month'])
df['war_events_pct_change'] = df.groupby('country_code')['war_events'].pct_change()
df['instability_change'] = df.groupby('country_code')['instability_index'].diff()

print("✅ Derived features added!")
print(f"   Total columns: {len(df.columns)}")
print(f"   Columns: {list(df.columns)}")

✅ Derived features added!
   Total columns: 33
   Columns: ['country_code', 'year_month', 'year', 'month', 'total_events', 'war_events', 'protest_events', 'sanctions_coercion_events', 'humanitarian_aid_events', 'threat_events', 'diplomatic_tension_events', 'force_posture_events', 'verbal_cooperation', 'material_cooperation', 'verbal_conflict', 'material_conflict', 'avg_goldstein_scale', 'min_goldstein_scale', 'max_goldstein_scale', 'avg_tone', 'min_tone', 'total_mentions', 'total_articles', 'total_sources', 'severe_negative_events', 'extreme_negative_events', 'avg_material_conflict_severity', 'conflict_ratio', 'instability_index', 'tone_negativity', 'media_attention_log', 'war_events_pct_change', 'instability_change']


In [14]:
# ---- CELL 6: Save to CSV (download from Colab) ----

output_filename = "gdelt_supply_chain_disruptions_2000_2024.csv"
df.to_csv(output_filename, index=False)

# Download to your local machine from Colab
from google.colab import files
files.download(output_filename)

print(f"\n✅ Saved & downloading: {output_filename}")
print(f"   File size: ~{df.memory_usage(deep=True).sum() / 1e6:.1f} MB in memory")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Saved & downloading: gdelt_supply_chain_disruptions_2000_2024.csv
   File size: ~24.7 MB in memory


In [38]:
import pandas as pd

gdelt = pd.read_csv("gdelt_supply_chain_disruptions_2000_2024.csv")

FIPS_TO_COUNTRY = {
    'AF':'Afghanistan','AL':'Albania','AG':'Algeria','AO':'Angola','AR':'Argentina',
    'AM':'Armenia','AS':'Australia','AU':'Austria','AJ':'Azerbaijan','BA':'Bahrain',
    'BG':'Bangladesh','BO':'Belarus','BE':'Belgium','BN':'Benin','BT':'Bhutan',
    'BL':'Bolivia','BK':'Bosnia','BC':'Botswana','BR':'Brazil','BX':'Brunei',
    'BU':'Bulgaria','UV':'Burkina Faso','BM':'Myanmar','BY':'Burundi','CB':'Cambodia',
    'CM':'Cameroon','CA':'Canada','CD':'Chad','CI':'Chile','CH':'China',
    'CO':'Colombia','CG':'Congo DRC','CF':'Congo Brazzaville','CS':'Costa Rica',
    'IV':'Ivory Coast','HR':'Croatia','CU':'Cuba','CY':'Cyprus','EZ':'Czech Republic',
    'DA':'Denmark','DJ':'Djibouti','DR':'Dominican Republic','EC':'Ecuador','EG':'Egypt',
    'ES':'El Salvador','ER':'Eritrea','EN':'Estonia','ET':'Ethiopia','FI':'Finland',
    'FR':'France','GB':'Gabon','GG':'Georgia','GM':'Germany','GH':'Ghana',
    'GR':'Greece','GT':'Guatemala','HA':'Haiti','HO':'Honduras','HU':'Hungary',
    'IC':'Iceland','IN':'India','ID':'Indonesia','IR':'Iran','IZ':'Iraq',
    'EI':'Ireland','IS':'Israel','IT':'Italy','JM':'Jamaica','JA':'Japan',
    'JO':'Jordan','KZ':'Kazakhstan','KE':'Kenya','KN':'North Korea','KS':'South Korea',
    'KU':'Kuwait','KG':'Kyrgyzstan','LA':'Laos','LG':'Latvia','LE':'Lebanon',
    'LT':'Lesotho','LI':'Liberia','LY':'Libya','LH':'Lithuania','MI':'Malawi',
    'MY':'Malaysia','ML':'Mali','MX':'Mexico','MN':'Mongolia','MO':'Morocco',
    'MZ':'Mozambique','WA':'Namibia','NP':'Nepal','NL':'Netherlands','NZ':'New Zealand',
    'NU':'Nicaragua','NG':'Niger','NI':'Nigeria','NO':'Norway','PK':'Pakistan',
    'PA':'Panama','PE':'Peru','RP':'Philippines','PL':'Poland','PO':'Portugal',
    'QA':'Qatar','RO':'Romania','RS':'Russia','RW':'Rwanda','SA':'Saudi Arabia',
    'SG':'Senegal','RI':'Serbia','SN':'Singapore','LO':'Slovakia','SI':'Slovenia',
    'SO':'Somalia','SF':'South Africa','OD':'South Sudan','SP':'Spain','CE':'Sri Lanka',
    'SU':'Sudan','SW':'Sweden','SZ':'Switzerland','SY':'Syria','TW':'Taiwan',
    'TZ':'Tanzania','TH':'Thailand','TS':'Tunisia','TU':'Turkey','UG':'Uganda',
    'UP':'Ukraine','AE':'UAE','UK':'United Kingdom','US':'United States',
    'UY':'Uruguay','UZ':'Uzbekistan','VE':'Venezuela','VM':'Vietnam','YM':'Yemen',
    'ZA':'Zambia','ZI':'Zimbabwe',
}

gdelt['country_name'] = gdelt['country_code'].map(FIPS_TO_COUNTRY)

# Check unmapped
unmapped = gdelt[gdelt['country_name'].isna()]['country_code'].unique()
print(f"✅ Mapped {gdelt['country_name'].notna().sum()}/{len(gdelt)} rows")
print(f"   Unmapped codes: {unmapped[:20]}")

# Save updated file
gdelt.to_csv("gdelt_supply_chain_disruptions_2000_2024.csv", index=False)
print(f"\n✅ Updated GDELT CSV with country_name column!")

✅ Mapped 41968/72748 rows
   Unmapped codes: ['AA' 'AC' 'AN' 'AQ' 'AV' 'AY' 'BB' 'BD' 'BF' 'BH' 'BP' 'BQ' 'BS' 'BV'
 'CJ' 'CN' 'CQ' 'CR' 'CT' 'CV']

✅ Updated GDELT CSV with country_name column!


In [39]:
FIPS_EXTRA = {
    'AA':'Aruba','AC':'Antigua and Barbuda','AN':'Andorra','AQ':'American Samoa',
    'AV':'Anguilla','AY':'Antarctica','BB':'Barbados','BD':'Bermuda',
    'BF':'Bahamas','BH':'Belize','BP':'Solomon Islands','BQ':'Navassa Island',
    'BS':'Bahamas','BV':'Bouvet Island','CJ':'Cayman Islands','CN':'Comoros',
    'CQ':'Northern Mariana Islands','CR':'Coral Sea Islands','CT':'Central African Republic',
    'CV':'Cape Verde','CW':'Cook Islands','DO':'Dominica','DX':'Dhekelia',
    'EK':'Equatorial Guinea','FG':'French Guiana','FJ':'Fiji','FK':'Falkland Islands',
    'FM':'Micronesia','FP':'French Polynesia','FQ':'Baker Island','GA':'Gambia',
    'GI':'Gibraltar','GJ':'Grenada','GK':'Guernsey','GL':'Greenland',
    'GP':'Guadeloupe','GQ':'Guam','GV':'Guinea','GZ':'Gaza Strip',
    'HK':'Hong Kong','HQ':'Howland Island','IO':'British Indian Ocean Territory',
    'IP':'Clipperton Island','JE':'Jersey','JN':'Jan Mayen','JQ':'Johnston Atoll',
    'KG':'Kyrgyzstan','KR':'Kiribati','KT':'Christmas Island','KV':'Kosovo',
    'LO':'Slovakia','LQ':'Palmyra Atoll','LS':'Liechtenstein','LU':'Luxembourg',
    'MA':'Madagascar','MB':'Martinique','MC':'Macau','MD':'Moldova',
    'MF':'Mayotte','MG':'Madagascar','MH':'Montserrat','MJ':'Montenegro',
    'MK':'North Macedonia','MM':'Myanmar','MP':'Mauritius','MQ':'Midway Islands',
    'MR':'Mauritania','MT':'Malta','MU':'Oman','MV':'Maldives',
    'NC':'New Caledonia','NE':'Niue','NF':'Norfolk Island','NH':'Vanuatu',
    'NS':'Suriname','NT':'Netherlands Antilles','PM':'Panama','PP':'Papua New Guinea',
    'PS':'Palau','PU':'Guinea-Bissau','RE':'Reunion','RM':'Marshall Islands',
    'RN':'Saint Martin','RQ':'Puerto Rico','SC':'Saint Kitts and Nevis',
    'SE':'Seychelles','SL':'Sierra Leone','SM':'San Marino','SN':'Singapore',
    'SP':'Spain','ST':'Saint Lucia','SV':'Svalbard','SX':'South Georgia',
    'TD':'Trinidad and Tobago','TI':'Tajikistan','TK':'Turks and Caicos',
    'TL':'Tokelau','TN':'Tonga','TO':'Togo','TP':'Sao Tome and Principe',
    'TT':'Timor-Leste','TV':'Tuvalu','TW':'Taiwan','TX':'Turkmenistan',
    'UC':'Curacao','UP':'Ukraine','VC':'Saint Vincent','VQ':'US Virgin Islands',
    'WE':'West Bank','WF':'Wallis and Futuna','WI':'Western Sahara',
    'WQ':'Wake Island','WS':'Samoa','WZ':'Eswatini',
}

gdelt['country_name'] = gdelt['country_name'].fillna(gdelt['country_code'].map(FIPS_EXTRA))

mapped = gdelt['country_name'].notna().sum()
print(f"✅ Now mapped: {mapped}/{len(gdelt)} ({mapped/len(gdelt)*100:.1f}%)")

unmapped = gdelt[gdelt['country_name'].isna()]['country_code'].unique()
print(f"   Still unmapped: {len(unmapped)} codes → {unmapped[:10]}")

gdelt.to_csv("gdelt_supply_chain_disruptions_2000_2024.csv", index=False)
print("✅ Saved!")

✅ Now mapped: 68775/72748 (94.5%)
   Still unmapped: 22 codes → ['DQ' 'EU' 'FO' 'GO' 'GY' 'HM' 'IM' 'JU' 'KQ' 'NR']
✅ Saved!


# Weather Data

In [24]:
!gcloud config set project resilience-ai-aatu
!gcloud services enable earthengine.googleapis.com

print("Waiting 30 seconds for it to propagate...")
import time
time.sleep(30)
print("✅ Done! Now re-run Cell 1 (the ee.Initialize cell)")

Updated property [core/project].
Operation "operations/acat.p2-130550627089-1f553899-77db-4ea3-8f25-1a523c7a659a" finished successfully.
Waiting 30 seconds for it to propagate...
✅ Done! Now re-run Cell 1 (the ee.Initialize cell)


In [26]:

# ============================================================
# CELL 1: Authenticate & Setup Earth Engine
# ============================================================

# Install Earth Engine API
!pip install earthengine-api -q

import ee
from google.colab import auth

# Authenticate
auth.authenticate_user()

# Initialize Earth Engine
# Use your same project from the GDELT pipeline
PROJECT_ID = "resilience-ai-aatu"  # <-- same project you used for GDELT

ee.Initialize(project=PROJECT_ID)

print("✅ Earth Engine initialized!")

✅ Earth Engine initialized!


In [27]:

# ============================================================
# CELL 2: Define country capitals (FIPS code, lat, lon)
# ============================================================

import pandas as pd
import numpy as np

# Same countries as before but as a list for Earth Engine
COUNTRIES = [
    # (FIPS, name, lat, lon, ISO3)
    ('AF', 'Afghanistan', 34.5553, 69.2075, 'AFG'),
    ('AL', 'Albania', 41.3275, 19.8187, 'ALB'),
    ('AG', 'Algeria', 36.7538, 3.0588, 'DZA'),
    ('AO', 'Angola', -8.8390, 13.2894, 'AGO'),
    ('AR', 'Argentina', -34.6037, -58.3816, 'ARG'),
    ('AM', 'Armenia', 40.1792, 44.4991, 'ARM'),
    ('AS', 'Australia', -35.2809, 149.1300, 'AUS'),
    ('AU', 'Austria', 48.2082, 16.3738, 'AUT'),
    ('AJ', 'Azerbaijan', 40.4093, 49.8671, 'AZE'),
    ('BA', 'Bahrain', 26.2285, 50.5860, 'BHR'),
    ('BG', 'Bangladesh', 23.8103, 90.4125, 'BGD'),
    ('BO', 'Belarus', 53.9006, 27.5590, 'BLR'),
    ('BE', 'Belgium', 50.8503, 4.3517, 'BEL'),
    ('BN', 'Benin', 6.4969, 2.6289, 'BEN'),
    ('BT', 'Bhutan', 27.4728, 89.6390, 'BTN'),
    ('BL', 'Bolivia', -16.4897, -68.1193, 'BOL'),
    ('BK', 'Bosnia', 43.8563, 18.4131, 'BIH'),
    ('BC', 'Botswana', -24.6282, 25.9231, 'BWA'),
    ('BR', 'Brazil', -15.8267, -47.9218, 'BRA'),
    ('BX', 'Brunei', 4.9031, 114.9398, 'BRN'),
    ('BU', 'Bulgaria', 42.6977, 23.3219, 'BGR'),
    ('UV', 'Burkina Faso', 12.3714, -1.5197, 'BFA'),
    ('BM', 'Burma/Myanmar', 19.7633, 96.0785, 'MMR'),
    ('BY', 'Burundi', -3.3614, 29.3599, 'BDI'),
    ('CB', 'Cambodia', 11.5564, 104.9282, 'KHM'),
    ('CM', 'Cameroon', 3.8480, 11.5021, 'CMR'),
    ('CA', 'Canada', 45.4215, -75.6972, 'CAN'),
    ('CD', 'Chad', 12.1348, 15.0557, 'TCD'),
    ('CI', 'Chile', -33.4489, -70.6693, 'CHL'),
    ('CH', 'China', 39.9042, 116.4074, 'CHN'),
    ('CO', 'Colombia', 4.7110, -74.0721, 'COL'),
    ('CG', 'Congo DRC', -4.4419, 15.2663, 'COD'),
    ('CF', 'Congo Brazzaville', -4.2634, 15.2429, 'COG'),
    ('CS', 'Costa Rica', 9.9281, -84.0907, 'CRI'),
    ('IV', 'Ivory Coast', 6.8276, -5.2893, 'CIV'),
    ('HR', 'Croatia', 45.8150, 15.9819, 'HRV'),
    ('CU', 'Cuba', 23.1136, -82.3666, 'CUB'),
    ('CY', 'Cyprus', 35.1856, 33.3823, 'CYP'),
    ('EZ', 'Czech Republic', 50.0755, 14.4378, 'CZE'),
    ('DA', 'Denmark', 55.6761, 12.5683, 'DNK'),
    ('DJ', 'Djibouti', 11.5880, 43.1456, 'DJI'),
    ('DR', 'Dominican Republic', 18.4861, -69.9312, 'DOM'),
    ('EC', 'Ecuador', -0.1807, -78.4678, 'ECU'),
    ('EG', 'Egypt', 30.0444, 31.2357, 'EGY'),
    ('ES', 'El Salvador', 13.6929, -89.2182, 'SLV'),
    ('ER', 'Eritrea', 15.3229, 38.9251, 'ERI'),
    ('EN', 'Estonia', 59.4370, 24.7536, 'EST'),
    ('ET', 'Ethiopia', 9.0250, 38.7469, 'ETH'),
    ('FI', 'Finland', 60.1699, 24.9384, 'FIN'),
    ('FR', 'France', 48.8566, 2.3522, 'FRA'),
    ('GB', 'Gabon', 0.4162, 9.4673, 'GAB'),
    ('GG', 'Georgia', 41.7151, 44.8271, 'GEO'),
    ('GM', 'Germany', 52.5200, 13.4050, 'DEU'),
    ('GH', 'Ghana', 5.6037, -0.1870, 'GHA'),
    ('GR', 'Greece', 37.9838, 23.7275, 'GRC'),
    ('GT', 'Guatemala', 14.6349, -90.5069, 'GTM'),
    ('HA', 'Haiti', 18.5944, -72.3074, 'HTI'),
    ('HO', 'Honduras', 14.0723, -87.1921, 'HND'),
    ('HU', 'Hungary', 47.4979, 19.0402, 'HUN'),
    ('IC', 'Iceland', 64.1466, -21.9426, 'ISL'),
    ('IN', 'India', 28.6139, 77.2090, 'IND'),
    ('ID', 'Indonesia', -6.2088, 106.8456, 'IDN'),
    ('IR', 'Iran', 35.6892, 51.3890, 'IRN'),
    ('IZ', 'Iraq', 33.3152, 44.3661, 'IRQ'),
    ('EI', 'Ireland', 53.3498, -6.2603, 'IRL'),
    ('IS', 'Israel', 31.7683, 35.2137, 'ISR'),
    ('IT', 'Italy', 41.9028, 12.4964, 'ITA'),
    ('JM', 'Jamaica', 18.1096, -77.2975, 'JAM'),
    ('JA', 'Japan', 35.6762, 139.6503, 'JPN'),
    ('JO', 'Jordan', 31.9454, 35.9284, 'JOR'),
    ('KZ', 'Kazakhstan', 51.1694, 71.4491, 'KAZ'),
    ('KE', 'Kenya', -1.2921, 36.8219, 'KEN'),
    ('KN', 'North Korea', 39.0392, 125.7625, 'PRK'),
    ('KS', 'South Korea', 37.5665, 126.9780, 'KOR'),
    ('KU', 'Kuwait', 29.3759, 47.9774, 'KWT'),
    ('LA', 'Laos', 17.9757, 102.6331, 'LAO'),
    ('LG', 'Latvia', 56.9496, 24.1052, 'LVA'),
    ('LE', 'Lebanon', 33.8938, 35.5018, 'LBN'),
    ('LI', 'Liberia', 6.2907, -10.7605, 'LBR'),
    ('LY', 'Libya', 32.8872, 13.1913, 'LBY'),
    ('LH', 'Lithuania', 54.6872, 25.2797, 'LTU'),
    ('MI', 'Malawi', -13.9626, 33.7741, 'MWI'),
    ('MY', 'Malaysia', 3.1390, 101.6869, 'MYS'),
    ('ML', 'Mali', 12.6392, -8.0029, 'MLI'),
    ('MX', 'Mexico', 19.4326, -99.1332, 'MEX'),
    ('MN', 'Mongolia', 47.8864, 106.9057, 'MNG'),
    ('MO', 'Morocco', 34.0209, -6.8416, 'MAR'),
    ('MZ', 'Mozambique', -25.9692, 32.5732, 'MOZ'),
    ('WA', 'Namibia', -22.5609, 17.0658, 'NAM'),
    ('NP', 'Nepal', 27.7172, 85.3240, 'NPL'),
    ('NL', 'Netherlands', 52.3676, 4.9041, 'NLD'),
    ('NZ', 'New Zealand', -41.2865, 174.7762, 'NZL'),
    ('NU', 'Nicaragua', 12.1150, -86.2362, 'NIC'),
    ('NG', 'Niger', 13.5127, 2.1128, 'NER'),
    ('NI', 'Nigeria', 9.0765, 7.3986, 'NGA'),
    ('NO', 'Norway', 59.9139, 10.7522, 'NOR'),
    ('PK', 'Pakistan', 33.6844, 73.0479, 'PAK'),
    ('PA', 'Panama', 8.9824, -79.5199, 'PAN'),
    ('PE', 'Peru', -12.0464, -77.0428, 'PER'),
    ('RP', 'Philippines', 14.5995, 120.9842, 'PHL'),
    ('PL', 'Poland', 52.2297, 21.0122, 'POL'),
    ('PO', 'Portugal', 38.7223, -9.1393, 'PRT'),
    ('QA', 'Qatar', 25.2854, 51.5310, 'QAT'),
    ('RO', 'Romania', 44.4268, 26.1025, 'ROU'),
    ('RS', 'Russia', 55.7558, 37.6173, 'RUS'),
    ('RW', 'Rwanda', -1.9403, 29.8739, 'RWA'),
    ('SA', 'Saudi Arabia', 24.7136, 46.6753, 'SAU'),
    ('SG', 'Senegal', 14.7167, -17.4677, 'SEN'),
    ('RI', 'Serbia', 44.7866, 20.4489, 'SRB'),
    ('SN', 'Singapore', 1.3521, 103.8198, 'SGP'),
    ('LO', 'Slovakia', 48.1486, 17.1077, 'SVK'),
    ('SI', 'Slovenia', 46.0569, 14.5058, 'SVN'),
    ('SO', 'Somalia', 2.0469, 45.3182, 'SOM'),
    ('SF', 'South Africa', -25.7479, 28.2293, 'ZAF'),
    ('SP', 'Spain', 40.4168, -3.7038, 'ESP'),
    ('CE', 'Sri Lanka', 6.9271, 79.8612, 'LKA'),
    ('SU', 'Sudan', 15.5007, 32.5599, 'SDN'),
    ('SW', 'Sweden', 59.3293, 18.0686, 'SWE'),
    ('SZ', 'Switzerland', 46.9480, 7.4474, 'CHE'),
    ('SY', 'Syria', 33.5138, 36.2765, 'SYR'),
    ('TW', 'Taiwan', 25.0330, 121.5654, 'TWN'),
    ('TZ', 'Tanzania', -6.7924, 39.2083, 'TZA'),
    ('TH', 'Thailand', 13.7563, 100.5018, 'THA'),
    ('TS', 'Tunisia', 36.8065, 10.1815, 'TUN'),
    ('TU', 'Turkey', 39.9334, 32.8597, 'TUR'),
    ('UG', 'Uganda', 0.3476, 32.5825, 'UGA'),
    ('UP', 'Ukraine', 50.4501, 30.5234, 'UKR'),
    ('AE', 'UAE', 24.4539, 54.3773, 'ARE'),
    ('UK', 'United Kingdom', 51.5074, -0.1278, 'GBR'),
    ('US', 'United States', 38.9072, -77.0369, 'USA'),
    ('UY', 'Uruguay', -34.9011, -56.1645, 'URY'),
    ('UZ', 'Uzbekistan', 41.2995, 69.2401, 'UZB'),
    ('VE', 'Venezuela', 10.4806, -66.9036, 'VEN'),
    ('VM', 'Vietnam', 21.0278, 105.8342, 'VNM'),
    ('YM', 'Yemen', 15.3694, 44.1910, 'YEM'),
    ('ZA', 'Zambia', -15.3875, 28.3228, 'ZMB'),
    ('ZI', 'Zimbabwe', -17.8292, 31.0522, 'ZWE'),
]

print(f"✅ {len(COUNTRIES)} countries loaded!")

✅ 137 countries loaded!


In [28]:
# ============================================================
# CELL 3: Extract ERA5 monthly data for all countries
# ============================================================

from tqdm import tqdm
import time

def get_era5_monthly_for_point(lat, lon, start_year=2000, end_year=2024):
    """
    Extract ERA5 monthly climate data for a single point.
    Uses Google Earth Engine — no rate limits!
    """
    point = ee.Geometry.Point([lon, lat])

    # ERA5 Monthly dataset in Earth Engine
    era5_monthly = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR') \
        .filterDate(f'{start_year}-01-01', f'{end_year}-12-31') \
        .select([
            'temperature_2m',              # Mean 2m temp (K)
            'dewpoint_temperature_2m',     # Dewpoint (K)
            'total_precipitation_sum',     # Total precip (m)
            'surface_pressure',            # Surface pressure (Pa)
            'u_component_of_wind_10m',     # U-wind (m/s)
            'v_component_of_wind_10m',     # V-wind (m/s)
            'total_evaporation_sum',       # Evaporation (m)
            'snowfall_sum',                # Snowfall (m)
            'snow_depth',                  # Snow depth (m)
            'soil_temperature_level_1',    # Soil temp (K)
        ])

    # Extract time series at the point
    def extract_values(image):
        values = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=11132  # ERA5-Land resolution ~11km
        )
        return image.set('values', values).set('date', image.date().format('YYYY-MM'))

    results = era5_monthly.map(extract_values)

    # Get as list
    info = results.aggregate_array('values').getInfo()
    dates = results.aggregate_array('date').getInfo()

    return info, dates


def process_era5_results(info, dates, fips, name, iso3):
    """Convert Earth Engine results to DataFrame."""
    rows = []
    for i, (values, date_str) in enumerate(zip(info, dates)):
        if values is None:
            continue

        year = int(date_str[:4])
        month = int(date_str[5:7])

        # Convert units
        temp_k = values.get('temperature_2m')
        temp_c = (temp_k - 273.15) if temp_k else None

        dewpoint_k = values.get('dewpoint_temperature_2m')
        dewpoint_c = (dewpoint_k - 273.15) if dewpoint_k else None

        precip_m = values.get('total_precipitation_sum')
        precip_mm = (precip_m * 1000) if precip_m else 0

        pressure_pa = values.get('surface_pressure')
        pressure_hpa = (pressure_pa / 100) if pressure_pa else None

        u_wind = values.get('u_component_of_wind_10m', 0)
        v_wind = values.get('v_component_of_wind_10m', 0)
        wind_speed = np.sqrt((u_wind or 0)**2 + (v_wind or 0)**2)

        evap_m = values.get('total_evaporation_sum')
        evap_mm = abs(evap_m * 1000) if evap_m else 0  # abs because evap is negative in ERA5

        snowfall_m = values.get('snowfall_sum')
        snowfall_mm = (snowfall_m * 1000) if snowfall_m else 0

        snow_depth = values.get('snow_depth', 0) or 0

        soil_temp_k = values.get('soil_temperature_level_1')
        soil_temp_c = (soil_temp_k - 273.15) if soil_temp_k else None

        rows.append({
            'country_code': fips,
            'country_name': name,
            'iso3': iso3,
            'year': year,
            'month': month,
            'year_month': year * 100 + month,
            'temp_mean': temp_c,
            'dewpoint_mean': dewpoint_c,
            'precip_total_mm': precip_mm,
            'pressure_hpa': pressure_hpa,
            'wind_speed_mean': wind_speed,
            'evapotranspiration_mm': evap_mm,
            'snowfall_mm': snowfall_mm,
            'snow_depth_m': snow_depth,
            'soil_temp_mean': soil_temp_c,
        })

    return pd.DataFrame(rows)


# ---- MAIN LOOP ----
all_weather = []
errors = []

print(f"Extracting ERA5 climate data for {len(COUNTRIES)} countries...")
print("Using Google Earth Engine — NO rate limits!\n")

for i, (fips, name, lat, lon, iso3) in enumerate(tqdm(COUNTRIES, desc="Countries")):
    try:
        info, dates = get_era5_monthly_for_point(lat, lon)
        df = process_era5_results(info, dates, fips, name, iso3)

        if not df.empty:
            all_weather.append(df)
            if (i + 1) % 20 == 0:
                print(f"  ✓ [{i+1}/{len(COUNTRIES)}] {name} — {len(df)} months")
        else:
            errors.append((fips, name))

    except Exception as e:
        errors.append((fips, name))
        if (i + 1) % 20 == 0:
            print(f"  ✗ [{i+1}/{len(COUNTRIES)}] {name} — {str(e)[:80]}")

    # Small pause to be nice (not required, but good practice)
    time.sleep(1)

weather_df = pd.concat(all_weather, ignore_index=True)

print(f"\n✅ ERA5 Weather data complete!")
print(f"   Shape: {weather_df.shape}")
print(f"   Countries: {weather_df['country_code'].nunique()}")
print(f"   Date range: {weather_df['year_month'].min()} to {weather_df['year_month'].max()}")
print(f"   Errors: {len(errors)} → {[e[1] for e in errors[:10]]}")

Extracting ERA5 climate data for 137 countries...
Using Google Earth Engine — NO rate limits!



Countries:  14%|█▍        | 19/137 [01:18<06:45,  3.43s/it]

  ✓ [20/137] Brunei — 300 months


Countries:  28%|██▊       | 39/137 [02:22<05:02,  3.09s/it]

  ✓ [40/137] Denmark — 300 months


...

  ✓ [100/137] Philippines — 300 months


Countries:  87%|████████▋ | 119/137 [06:19<00:51,  2.85s/it]

  ✓ [120/137] Syria — 300 months


Countries: 100%|██████████| 137/137 [07:13<00:00,  3.17s/it]


✅ ERA5 Weather data complete!
   Shape: (41100, 15)
   Countries: 137
   Date range: 200001 to 202412
   Errors: 0 → []


In [29]:

# ============================================================
# CELL 4: Add derived features & anomalies
# ============================================================

print("Computing derived features and anomalies...")

# Humidity proxy from dewpoint
weather_df['humidity_proxy'] = weather_df['dewpoint_mean'] / weather_df['temp_mean'].clip(lower=0.1)

# Drought index: evaporation vs precipitation
weather_df['drought_index'] = (
    weather_df['evapotranspiration_mm'] /
    weather_df['precip_total_mm'].clip(lower=1)
)

# Heat stress indicator
weather_df['heat_stress'] = (
    (weather_df['temp_mean'] > 30).astype(int) * 0.5 +
    (weather_df['humidity_proxy'] > 0.8).astype(int) * 0.5
)

# Compute anomalies (2000-2019 baseline)
baseline = weather_df[weather_df['year'].between(2000, 2019)].groupby(
    ['country_code', 'month']
).agg(
    baseline_temp=('temp_mean', 'mean'),
    baseline_temp_std=('temp_mean', 'std'),
    baseline_precip=('precip_total_mm', 'mean'),
    baseline_precip_std=('precip_total_mm', 'std'),
).reset_index()

weather_df = weather_df.merge(baseline, on=['country_code', 'month'], how='left')

weather_df['temp_anomaly'] = weather_df['temp_mean'] - weather_df['baseline_temp']
weather_df['temp_anomaly_zscore'] = (
    (weather_df['temp_mean'] - weather_df['baseline_temp']) /
    weather_df['baseline_temp_std'].clip(lower=0.1)
)
weather_df['precip_anomaly'] = weather_df['precip_total_mm'] - weather_df['baseline_precip']
weather_df['precip_anomaly_zscore'] = (
    (weather_df['precip_total_mm'] - weather_df['baseline_precip']) /
    weather_df['baseline_precip_std'].clip(lower=0.1)
)

# Composite weather severity score
weather_df['weather_severity'] = (
    weather_df['temp_anomaly_zscore'].abs() * 0.35 +
    weather_df['precip_anomaly_zscore'].abs() * 0.35 +
    weather_df['drought_index'].clip(0, 5) / 5 * 0.15 +
    weather_df['heat_stress'] * 0.15
).clip(0, 1)

# Drop baseline helper columns
weather_df = weather_df.drop(columns=[
    'baseline_temp', 'baseline_temp_std',
    'baseline_precip', 'baseline_precip_std'
])

print(f"✅ Features computed! Total columns: {len(weather_df.columns)}")
print(f"   Columns: {list(weather_df.columns)}")

Computing derived features and anomalies...
✅ Features computed! Total columns: 23
   Columns: ['country_code', 'country_name', 'iso3', 'year', 'month', 'year_month', 'temp_mean', 'dewpoint_mean', 'precip_total_mm', 'pressure_hpa', 'wind_speed_mean', 'evapotranspiration_mm', 'snowfall_mm', 'snow_depth_m', 'soil_temp_mean', 'humidity_proxy', 'drought_index', 'heat_stress', 'temp_anomaly', 'temp_anomaly_zscore', 'precip_anomaly', 'precip_anomaly_zscore', 'weather_severity']


In [30]:

# ============================================================
# CELL 5: Validate
# ============================================================

print("📊 Sample Data:")
print(weather_df.head(10))

print("\n📊 Sanity Checks:")
india = weather_df[(weather_df['country_code'] == 'IN') & (weather_df['month'] == 5)]
print(f"India May avg temp: {india['temp_mean'].mean():.1f}°C (expect ~30-35°C)")

norway = weather_df[(weather_df['country_code'] == 'NO') & (weather_df['month'] == 1)]
print(f"Norway Jan avg temp: {norway['temp_mean'].mean():.1f}°C (expect ~-3 to -5°C)")

brazil = weather_df[(weather_df['country_code'] == 'BR') & (weather_df['month'] == 1)]
print(f"Brazil Jan precip: {brazil['precip_total_mm'].mean():.1f}mm (expect ~100-250mm)")

📊 Sample Data:
  country_code country_name iso3  year  month  year_month  temp_mean  \
0           AF  Afghanistan  AFG  2000      1      200001  -1.042170   
1           AF  Afghanistan  AFG  2000      2      200002  -4.732696   
2           AF  Afghanistan  AFG  2000      3      200003   4.633834   
3           AF  Afghanistan  AFG  2000      4      200004  17.045522   
4           AF  Afghanistan  AFG  2000      5      200005  24.471698   
5           AF  Afghanistan  AFG  2000      6      200006  25.241889   
6           AF  Afghanistan  AFG  2000      7      200007  27.029182   
7           AF  Afghanistan  AFG  2000      8      200008  26.126145   
8           AF  Afghanistan  AFG  2000      9      200009  22.520120   
9           AF  Afghanistan  AFG  2000     10      200010  17.099301   

   dewpoint_mean  precip_total_mm  pressure_hpa  ...  snow_depth_m  \
0      -9.408451        46.751075    838.840912  ...  1.591561e-01   
1     -13.675946        35.384255    838.305539  ...

In [31]:
# ============================================================
# CELL 6: Save & Download
# ============================================================

filename = "weather_data_2000_2024.csv"
weather_df.to_csv(filename, index=False)

from google.colab import files
files.download(filename)

# Google Drive backup
from google.colab import drive
import os
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/ResilienceAI/data/'
os.makedirs(drive_path, exist_ok=True)
weather_df.to_csv(f'{drive_path}{filename}', index=False)

print(f"\n✅ Weather data saved!")
print(f"   Shape: {weather_df.shape}")
print(f"   Countries: {weather_df['country_code'].nunique()}")
print(f"   File: {filename}")

print("""
╔════════════════════════════════════════════════════════════╗
║  OUTPUT — Ready to join with GDELT                        ║
║                                                            ║
║  Join keys: country_code + year_month                      ║
║                                                            ║
║  Weather columns include:                                  ║
║  • temp_mean, dewpoint_mean, precip_total_mm               ║
║  • wind_speed_mean, evapotranspiration_mm                  ║
║  • snowfall_mm, snow_depth_m, soil_temp_mean               ║
║  • pressure_hpa, humidity_proxy                            ║
║  • drought_index, heat_stress                              ║
║  • temp_anomaly, temp_anomaly_zscore                       ║
║  • precip_anomaly, precip_anomaly_zscore                   ║
║  • weather_severity (composite 0-1)                        ║
╚════════════════════════════════════════════════════════════╝
""")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mounted at /content/drive

✅ Weather data saved!
   Shape: (41100, 23)
   Countries: 137
   File: weather_data_2000_2024.csv

╔════════════════════════════════════════════════════════════╗
║  OUTPUT — Ready to join with GDELT                        ║
║                                                            ║
║  Join keys: country_code + year_month                      ║
║                                                            ║
║  Weather columns include:                                  ║
║  • temp_mean, dewpoint_mean, precip_total_mm               ║
║  • wind_speed_mean, evapotranspiration_mm                  ║
║  • snowfall_mm, snow_depth_m, soil_temp_mean               ║
║  • pressure_hpa, humidity_proxy                            ║
║  • drought_index, heat_stress                              ║
║  • temp_anomaly, temp_anomaly_zscore                       ║
║  • precip_anomaly, precip_anomaly_zscore                   ║
║  • weather_severity (composite 0-1)                   

# EM-DAT Disaster Data

In [33]:
# EM-DAT from HDX — fetch the actual download link first

import requests

# Get the dataset page to find current resource URL
hdx_api = "https://data.humdata.org/api/3/action/package_show?id=emdat-country-profiles"
resp = requests.get(hdx_api)
data = resp.json()

# Find the download URL from the resources
for resource in data['result']['resources']:
    print(f"Name: {resource['name']}")
    print(f"URL: {resource['url']}")
    print(f"Format: {resource.get('format', 'N/A')}")
    print("---")

Name: EMDAT-country-profiles_2026_02_13.xlsx
URL: https://data.humdata.org/dataset/74163686-a029-4e27-8fbf-c5bfcd13f953/resource/c5ce40d6-07b1-4f36-955a-d6196436ff6b/download/emdat-country-profiles_2026_02_13.xlsx
Format: XLSX
---


In [34]:
EMDAT_URL = "https://data.humdata.org/dataset/74163686-a029-4e27-8fbf-c5bfcd13f953/resource/c5ce40d6-07b1-4f36-955a-d6196436ff6b/download/emdat-country-profiles_2026_02_13.xlsx"

import requests
import pandas as pd

resp = requests.get(EMDAT_URL)
with open("emdat_raw.xlsx", "wb") as f:
    f.write(resp.content)

emdat_df = pd.read_excel("emdat_raw.xlsx")
print(f"✅ Downloaded! Shape: {emdat_df.shape}")
print(f"\nColumns: {list(emdat_df.columns)}")
print(f"\n{emdat_df.head()}")

✅ Downloaded! Shape: (6432, 13)

Columns: ['Year', 'Country', 'ISO', 'Disaster Group', 'Disaster Subroup', 'Disaster Type', 'Disaster Subtype', 'Total Events', 'Total Affected', 'Total Deaths', 'Total Damage (USD, original)', 'Total Damage (USD, adjusted)', 'CPI']

              Year         Country             ISO Disaster Group  \
0  #date +occurred  #country +name  #country +code  #cause +group   
1             2000     Afghanistan             AFG        Natural   
2             2000         Algeria             DZA        Natural   
3             2000          Angola             AGO        Natural   
4             2000          Angola             AGO        Natural   

   Disaster Subroup Disaster Type Disaster Subtype Total Events  \
0  #cause +subgroup  #cause +type  #cause +subtype   #frequency   
1    Climatological       Drought          Drought            1   
2      Hydrological         Flood      Flash flood            1   
3      Hydrological         Flood  Flood (General) 

In [36]:
# ============================================================
# CELL 2: Clean & Process EM-DAT
# ============================================================

# Row 0 is HXL tags, drop it
emdat_df = emdat_df.iloc[1:].reset_index(drop=True)

# Convert types
emdat_df['Year'] = pd.to_numeric(emdat_df['Year'], errors='coerce').astype('Int64')
emdat_df['Total Events'] = pd.to_numeric(emdat_df['Total Events'], errors='coerce').astype('Int64')
emdat_df['Total Affected'] = pd.to_numeric(emdat_df['Total Affected'], errors='coerce')
emdat_df['Total Deaths'] = pd.to_numeric(emdat_df['Total Deaths'], errors='coerce')
emdat_df['Total Damage (USD, adjusted)'] = pd.to_numeric(emdat_df['Total Damage (USD, adjusted)'], errors='coerce')

# Filter 2000-2024
emdat_df = emdat_df[(emdat_df['Year'] >= 2000) & (emdat_df['Year'] <= 2024)]

# Map ISO3 → FIPS
ISO3_TO_FIPS = {
    'AFG':'AF','ALB':'AL','DZA':'AG','AGO':'AO','ARG':'AR','ARM':'AM',
    'AUS':'AS','AUT':'AU','AZE':'AJ','BHR':'BA','BGD':'BG','BLR':'BO',
    'BEL':'BE','BEN':'BN','BTN':'BT','BOL':'BL','BIH':'BK','BWA':'BC',
    'BRA':'BR','BRN':'BX','BGR':'BU','BFA':'UV','MMR':'BM','BDI':'BY',
    'KHM':'CB','CMR':'CM','CAN':'CA','TCD':'CD','CHL':'CI','CHN':'CH',
    'COL':'CO','COD':'CG','COG':'CF','CRI':'CS','CIV':'IV','HRV':'HR',
    'CUB':'CU','CYP':'CY','CZE':'EZ','DNK':'DA','DJI':'DJ','DOM':'DR',
    'ECU':'EC','EGY':'EG','SLV':'ES','ERI':'ER','EST':'EN','ETH':'ET',
    'FIN':'FI','FRA':'FR','GAB':'GB','GEO':'GG','DEU':'GM','GHA':'GH',
    'GRC':'GR','GTM':'GT','HTI':'HA','HND':'HO','HUN':'HU','ISL':'IC',
    'IND':'IN','IDN':'ID','IRN':'IR','IRQ':'IZ','IRL':'EI','ISR':'IS',
    'ITA':'IT','JAM':'JM','JPN':'JA','JOR':'JO','KAZ':'KZ','KEN':'KE',
    'PRK':'KN','KOR':'KS','KWT':'KU','KGZ':'KG','LAO':'LA','LVA':'LG',
    'LBN':'LE','LSO':'LT','LBR':'LI','LBY':'LY','LTU':'LH','MWI':'MI',
    'MYS':'MY','MLI':'ML','MEX':'MX','MNG':'MN','MAR':'MO','MOZ':'MZ',
    'NAM':'WA','NPL':'NP','NLD':'NL','NZL':'NZ','NIC':'NU','NER':'NG',
    'NGA':'NI','NOR':'NO','PAK':'PK','PAN':'PA','PER':'PE','PHL':'RP',
    'POL':'PL','PRT':'PO','QAT':'QA','ROU':'RO','RUS':'RS','RWA':'RW',
    'SAU':'SA','SEN':'SG','SRB':'RI','SGP':'SN','SVK':'LO','SVN':'SI',
    'SOM':'SO','ZAF':'SF','SSD':'OD','ESP':'SP','LKA':'CE','SDN':'SU',
    'SWE':'SW','CHE':'SZ','SYR':'SY','TWN':'TW','TZA':'TZ','THA':'TH',
    'TUN':'TS','TUR':'TU','UGA':'UG','UKR':'UP','ARE':'AE','GBR':'UK',
    'USA':'US','URY':'UY','UZB':'UZ','VEN':'VE','VNM':'VM','YEM':'YM',
    'ZMB':'ZA','ZWE':'ZI',
}

emdat_df['country_code'] = emdat_df['ISO'].map(ISO3_TO_FIPS)

print(f"✅ Processed!")
print(f"   Shape: {emdat_df.shape}")
print(f"   Years: {emdat_df['Year'].min()} to {emdat_df['Year'].max()}")
print(f"   Countries: {emdat_df['ISO'].nunique()}")
print(f"   FIPS mapped: {emdat_df['country_code'].notna().sum()}/{len(emdat_df)}")
print(f"\n📊 Disaster types:")
print(emdat_df['Disaster Type'].value_counts().head(10))
print(f"\n📊 Sample:")
print(emdat_df.head())

✅ Processed!
   Shape: (6202, 14)
   Years: 2000 to 2024
   Countries: 217
   FIPS mapped: 5495/6202

📊 Disaster types:
Disaster Type
Flood                  2498
Storm                  1624
Extreme temperature     527
Drought                 404
Earthquake              398
Mass movement (wet)     347
Wildfire                246
Volcanic activity       109
Infestation              29
Mass movement (dry)      14
Name: count, dtype: int64

📊 Sample:
   Year      Country  ISO Disaster Group Disaster Subroup  \
0  2000  Afghanistan  AFG        Natural   Climatological   
1  2000      Algeria  DZA        Natural     Hydrological   
2  2000       Angola  AGO        Natural     Hydrological   
3  2000       Angola  AGO        Natural     Hydrological   
4  2000       Angola  AGO        Natural     Hydrological   

         Disaster Type Disaster Subtype  Total Events  Total Affected  \
0              Drought          Drought             1       2580000.0   
1                Flood      Flash fl

In [37]:
# ============================================================
# CELL 3: Save EM-DAT
# ============================================================

emdat_filename = "emdat_disasters_2000_2024.csv"
emdat_df.to_csv(emdat_filename, index=False)

from google.colab import files
files.download(emdat_filename)

# Google Drive backup
from google.colab import drive
import os
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/ResilienceAI/data/'
os.makedirs(drive_path, exist_ok=True)
emdat_df.to_csv(f'{drive_path}{emdat_filename}', index=False)

print(f"✅ EM-DAT saved: {emdat_filename}")
print(f"   Shape: {emdat_df.shape}")

print("""
╔═══════════════════════════════════════════════════════╗
║  🎉 ALL 3 DATASETS COMPLETE!                         ║
║                                                       ║
║  1. ✅ GDELT (news)    → 72,748 rows, 33 cols        ║
║  2. ✅ ERA5 (weather)  → 41,100 rows, 23 cols        ║
║  3. ✅ EM-DAT (disaster) → saved!                    ║
║                                                       ║
║  Ready to merge on country_code + year_month/year     ║
╚═══════════════════════════════════════════════════════╝
""")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ EM-DAT saved: emdat_disasters_2000_2024.csv
   Shape: (6202, 14)

╔═══════════════════════════════════════════════════════╗
║  🎉 ALL 3 DATASETS COMPLETE!                         ║
║                                                       ║
║  1. ✅ GDELT (news)    → 72,748 rows, 33 cols        ║
║  2. ✅ ERA5 (weather)  → 41,100 rows, 23 cols        ║
║  3. ✅ EM-DAT (disaster) → saved!                    ║
║                                                       ║
║  Ready to merge on country_code + year_month/year     ║
╚═══════════════════════════════════════════════════════╝

